In [1]:
# print("Hello world!")
# install.packages("colorspace")
# install.packages("data.table")
# install.packages("plyr")
# install.packages("dplyr")
# install.packages("optparse")
# install.packages("bit64")

#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install()
# BiocManager::install("rhdf5")

In [2]:
setwd("/frazer01/home/aphodges/software/my_hla_typing/")
source("script/functions.R")

In [3]:

dir()

[1] "1kgp"                  "gen.hla_types_all.txt" "gen.hla_types.txt"    
 [4] "gen.result.txt"        "hla_typing.sh"         "HLAVBSeq.jar"         
 [7] "input"                 "nohup.out"             "pipeline"             
[10] "process_hla_types.R"   "README.md"             "reference"            
[13] "script"                "TMP"

In [4]:
id2url_2504           = fread("input/1000G_2504_high_coverage.sequence.index"        , sep = "\t", header = TRUE , data.table = FALSE)[,c("#ENA_FILE_PATH", "RUN_ID", "LIBRARY_NAME", "POPULATION")]
id2url_698            = fread("input/1000G_698_related_high_coverage.sequence.index" , sep = "\t", header = FALSE, data.table = FALSE)[,c(1,3,10,11)]
colnames(id2url_2504) = c("cram_url", "run_id", "id", "population")
colnames(id2url_698 ) = c("cram_url", "run_id", "id", "population")
id2url                = rbind(id2url_2504, id2url_698)

fwrite(id2url, "pipeline/hla_typing/id2url.txt", sep = "\t", col.names = TRUE, row.names = FALSE)


Warning message in require_bit64_if_needed(ans):
“Some columns are type 'integer64' but package bit64 is not installed. Those columns will print as strange looking floating point data. There is no need to reload the data. Simply install.packages('bit64') to obtain the integer64 print method and print the data again.”
Warning message in require_bit64_if_needed(ans):
“Some columns are type 'integer64' but package bit64 is not installed. Those columns will print as strange looking floating point data. There is no need to reload the data. Simply install.packages('bit64') to obtain the integer64 print method and print the data again.”


In [6]:
run_hla_typing_qsub = function(id2url, qsub = FALSE)
{
    sh_file = paste(getwd(), "pipeline/hla_typing/run_hla_typing.sh", sep = "/")
    
    writeLines(text = c("#!/usr/bin/sh",
                        "source /frazer01/home/aphodges/.bashrc",
                        "export REF_PATH=/frazer01/home/aphodges/software/my_hla_typing/input/cram_cache/cache/%2s/%2s/%s:http://www.ebi.ac.uk/ena/cram/md5/%s","
                        export REF_CACHE=/frazer01/home/aphodges/software/my_hla_typing/input/cram_cache/cache/%2s/%2s/%s",
                        #"export REF_PATH=/frazer01/projects/CEGS/analysis/hla_type_1kgp/input/cram_cache/cache/%2s/%2s/%s:http://www.ebi.ac.uk/ena/cram/md5/%s",
                        #"export REF_CACHE=/frazer01/projects/CEGS/analysis/hla_type_1kgp/input/cram_cache/cache/%2s/%2s/%s",
                        paste("Rscript", paste(paste(getwd(), "script", "run_hla_typing.R", sep = "/"),
                                               "--taskid"   , "$SGE_TASK_ID")
                             )
                       ), 
               con  = sh_file, 
               sep  = "\n\n")
    
    qsub_command = paste("qsub",
                         "-t", paste(1, "-", 50, ":1", sep = ""), #5 replacing nrow(id2url)
                         "-tc", 50, 
                         "-l" , "short",
                         "-l" , "h_vmem=16G",
                         "-pe", "smp", 8, #8 set previously
                         "-o" , paste(getwd(), "pipeline/hla_typing/logs.out", sep = "/"),
                         "-e" , paste(getwd(), "pipeline/hla_typing/logs.err", sep = "/"),
                         sh_file
                        )
    
    message(qsub_command)
    if(qsub == TRUE){system(qsub_command)}
}

run_hla_typing_qsub(id2url, TRUE)


qsub -t 1-50:1 -tc 50 -l short -l h_vmem=16G -pe smp 8 -o /frazer01/home/aphodges/software/my_hla_typing/pipeline/hla_typing/logs.out -e /frazer01/home/aphodges/software/my_hla_typing/pipeline/hla_typing/logs.err /frazer01/home/aphodges/software/my_hla_typing/pipeline/hla_typing/run_hla_typing.sh



In [3]:
error_data = read.table("pipeline/hla_typing/logs.out")

ERROR: Error in read.table("pipeline/hla_typing/logs.out"): no lines available in input


In [11]:
print(error_data[1,])

ERROR: Error in print(error_data[1, ]): object 'error_data' not found


In [17]:
system("qsub -t 1-5:1 -tc 50 -l short -l h_vmem=8G -pe smp 2 -o /frazer01/home/aphodges/software/my_hla_typing/pipeline/hla_typing/logs.out -e /frazer01/home/aphodges/software/my_hla_typing/pipeline/hla_typing/logs.err \
/frazer01/home/aphodges/software/my_hla_typing/pipeline/hla_typing/run_hla_typing.sh")